In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from dataloader_multimodal import VideoDataset

In [2]:
class AuralSkillClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        # inicia um backbone da rede com a ResNet18 pré-treinada
        self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)

        # modifica a primeira camada para receber apenas 1 canal de entrada
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        # remove a camada de classificação original da ResNet, substituindo-a por uma nn.Identity()
        self.backbone.fc = nn.Identity()

        # cria uma nova camada de classificação, conforme descrita no artigo, com 512 valores de entrada e 128 de saída,
        # posteriormente passando por uma camada que faz a classificação de fato, com 10 valores de saída (correspondentes às classes)
        self.classifier = nn.Sequential(
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        self.classification_head = nn.Linear(128, num_classes)
        self.regression_head = nn.Linear(128, 1)

    def forward(self, x):
        features = self.backbone(x) # passa a entrada pela rede ResNet
        features_128 = self.classifier(features) # passa o resultado da ResNet pela camada final de classificação
        logits_cls = self.classification_head(features_128)
        output_reg = self.regression_head(features_128)

        return logits_cls, output_reg

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo: {device}")

# 2. Crie uma instância do seu modelo e mova para o dispositivo
model = AuralSkillClassifier(num_classes=10).to(device)

# Coloca o modelo em modo de avaliação (importante para desativar o Dropout)
model.eval()

# 3. Crie um lote de dados "falsos" (dummy tensor)
# Formato: [batch_size, channels, height, width]
# batch_size = 4 (simulando 4 imagens de uma vez)
# channels = 1 (seu espectrograma é escala de cinza)
# height = 224
# width = 224
batch_size = 4
dummy_input = torch.rand(batch_size, 1, 224, 224).to(device)

print(f"Formato do tensor de entrada: {dummy_input.shape}")

# 4. Passe os dados falsos pelo modelo
# torch.no_grad() desliga o cálculo de gradientes, o que torna
# a inferência mais rápida e economiza memória.
with torch.no_grad():
    output = model(dummy_input)

# 5. Verifique a saída
print(f"Formato do tensor de saída: {output[0].shape}, {output[1].shape}")
print("Tensor de saída (logits):")
print(output)

Usando o dispositivo: cuda
Formato do tensor de entrada: torch.Size([4, 1, 224, 224])
Formato do tensor de saída: torch.Size([4, 10]), torch.Size([4, 1])
Tensor de saída (logits):
(tensor([[ 0.0240,  0.1101,  0.2270,  0.3069,  0.0485,  0.0774, -0.1472,  0.3770,
         -0.1193,  0.0270],
        [ 0.0271,  0.1411,  0.2155,  0.3088,  0.0610,  0.0791, -0.1510,  0.3970,
         -0.1299,  0.0367],
        [ 0.0285,  0.1314,  0.2266,  0.3071,  0.0486,  0.0897, -0.1352,  0.3748,
         -0.1330,  0.0397],
        [ 0.0255,  0.1292,  0.2180,  0.3128,  0.0572,  0.0897, -0.1448,  0.3905,
         -0.1271,  0.0361]], device='cuda:0'), tensor([[0.0479],
        [0.0481],
        [0.0583],
        [0.0589]], device='cuda:0'))


In [4]:
criterion_cls = nn.CrossEntropyLoss()
criterion_reg_l1 = nn.L1Loss()
criterion_reg_l2 = nn.MSELoss()

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [6]:
train_dataset = VideoDataset(mode='train')

In [7]:
# Crie o DataLoader (você já deve ter feito isso em um passo anterior)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

In [8]:
from opts import *

In [9]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"--- Iniciando Época {epoch+1}/{num_epochs} ---")
    # para cada lote de dados:
    # pegar o tensor 5D de espectrogramas

    # criar listas vazias para guardar as saídas

    # para cada um dos 10 clipes:
    # fatiar o tensor 5D para pegar o clipe atual (que será 4D)
    # passar o clipe 4D pelo modelo
    # guardar as duas saídas (cls e reg) nas listas

    # agregar os resultados das listas (ex: empilhar e tirar a média)

    # usar as saídas médias para calcular a perda total (cls + reg)

    # fazer loss.backward()
    # fazer optimizer.step()
    for i, batch_data in enumerate(train_loader):
        print(f"  Processando lote {i+1}/{len(train_loader)}")
        optimizer.zero_grad()
        spectrograms_tensor = batch_data['audio'].to(device)
        labels = batch_data['player_lvl'].to(device)
        lista_outputs_cls = []
        lista_outputs_reg = []
        labels_long = labels.long()
        labels_float = labels.float()

        for i in range(nclips):
            clip_tensor = spectrograms_tensor[:, i, :, :, :]
            logits_cls_clip, output_reg_clip = model(clip_tensor)
            lista_outputs_cls.append(logits_cls_clip)
            lista_outputs_reg.append(output_reg_clip)
        logits_cls = (torch.stack(lista_outputs_cls)).mean(dim=0)
        output_reg = (torch.stack(lista_outputs_reg)).mean(dim=0)
        
        loss_cls = criterion_cls(logits_cls, labels_long)

        output_reg = output_reg.squeeze()
        l1 = criterion_reg_l1(output_reg, labels_float)
        l2 = criterion_reg_l2(output_reg, labels_float)
        loss_reg = l1 + l2
        loss = (1.0 * loss_cls) + (0.1 * loss_reg)
        loss.backward()
        optimizer.step()
    print(f"--- Época {epoch+1} concluída ---")

--- Iniciando Época 1/10 ---
  Processando lote 1/129
  Processando lote 2/129
  Processando lote 3/129
  Processando lote 4/129
  Processando lote 5/129
  Processando lote 6/129
  Processando lote 7/129
  Processando lote 8/129
  Processando lote 9/129
  Processando lote 10/129
  Processando lote 11/129
  Processando lote 12/129
  Processando lote 13/129
  Processando lote 14/129
  Processando lote 15/129
  Processando lote 16/129
  Processando lote 17/129
  Processando lote 18/129
  Processando lote 19/129
  Processando lote 20/129
  Processando lote 21/129
  Processando lote 22/129
  Processando lote 23/129
  Processando lote 24/129
  Processando lote 25/129
  Processando lote 26/129
  Processando lote 27/129
  Processando lote 28/129
  Processando lote 29/129
  Processando lote 30/129
  Processando lote 31/129
  Processando lote 32/129
  Processando lote 33/129
  Processando lote 34/129
  Processando lote 35/129
  Processando lote 36/129
  Processando lote 37/129
  Processando lote

KeyboardInterrupt: 

In [ ]:
# --- Bloco de Código para Visualização ---
import matplotlib.pyplot as plt

# Supondo que você já tenha criado seu 'train_dataset' assim:
# train_dataset = VideoDataset(mode='train')

print("Carregando uma amostra do dataset para visualização...")

# 1. Pegue uma amostra do dataset (aqui, pegamos a primeira, índice 0)
# O __getitem__ do seu dataset será chamado e retornará o dicionário de dados.
sample_data = train_dataset[0]

# 2. Extraia o tensor dos espectrogramas e a etiqueta (label)
# Lembre-se que 'audio' é um tensor com shape [nclips, C, H, W], ou seja, [10, 1, 224, 224]
spectrograms_tensor = sample_data['audio']
label = sample_data['player_lvl']

print(f"Formato do tensor de áudio da amostra: {spectrograms_tensor.shape}")
print(f"Nível de Habilidade (Label): {label}")

# 3. Escolha um dos 10 clipes para visualizar (vamos pegar o primeiro, índice 0)
first_clip_tensor = spectrograms_tensor[0]

# 4. Prepare o tensor para ser plotado com matplotlib
#    - .squeeze() remove dimensões de tamanho 1. Transforma [1, 224, 224] em [224, 224].
#    - .cpu() garante que o tensor está na CPU, pois matplotlib não funciona com tensores na GPU.
#    - .numpy() converte o tensor do PyTorch para um array do NumPy, que é o formato que o matplotlib espera.
image_to_show = first_clip_tensor.squeeze().cpu().numpy()

# 5. Plote a imagem
plt.figure(figsize=(8, 8))
plt.title(f"Espectrograma do Clipe 0 | Nível de Habilidade: {label}")
plt.xlabel("Tempo (esticado para 224 pixels)")
plt.ylabel("Frequência (esticada para 224 pixels)")
# cmap='gray' garante que a imagem seja exibida em escala de cinza
plt.imshow(image_to_show, cmap='gray', origin='lower') 
plt.colorbar(label='Intensidade (normalizada)') # Adiciona uma barra de cores para indicar a intensidade
plt.show()